## setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo",
              "api_key": OPENAI_API_KEY}

## task

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unveiling the World of DeepLearning.AI"

Embark on a journey to unravel the mysteries of AI with DeepLearning.AI! Offering cutting-edge online courses created by the renowned AI expert Andrew Ng, this platform equips learners with the knowledge and skills to thrive in the age of artificial intelligence. From deep learning fundamentals to advanced applications, DeepLearning.AI caters to beginners and experts alike. Join a global community of learners, upskill in-demand AI expertise, and pave the way for a successful career in this rapidly evolving field. Dive into DeepLearning.AI today and unleash your potential in the world of artificial intelligence!


## two-agents (actor-critic) reflection

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of Deep Learning with DeepLearning.AI"

Dive into the revolutionary world of AI with DeepLearning.AI! As a leading platform founded by Andrew Ng, a pioneer in the field of deep learning, this online hub offers top-tier courses to master the art of artificial intelligence. Whether you're a novice or an expert, DeepLearning.AI provides the tools and knowledge to succeed in this rapidly evolving field. From neural networks to computer vision, embark on a learning journey that will enhance your skills and open doors to endless possibilities. Join the DeepLearning.AI community today and unlock the potential of AI!

--------------------------------------------------------------------------------
Critic (to Wr

## nested chats

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## orchestrate nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of Deep Learning with DeepLearning.AI"

Dive into the revolutionary world of AI with DeepLearning.AI! As a leading platform founded by Andrew Ng, a pioneer in the field of deep learning, this online hub offers top-tier courses to master the art of artificial intelligence. Whether you're a novice or an expert, DeepLearning.AI provides the tools and knowledge to succeed in this rapidly evolving field. From neural networks to computer vision, embark on a learning journey that will enhance your skills and open doors to endless possibilities. Join the DeepLearning.AI community today and unlock the potential of AI!

--------------------------------------------------------------------------------

************

/home/akash/miniconda3/envs/py310/lib/python3.10/site-packages/autogen/agentchat/conversable_agent.py:1315: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

As a legal reviewer, I suggest the following for the content provided:

- Ensure that the use of the term "top-tier courses" is substantiated with concrete evidence to avoid potential claims of false advertising.

- Consider including a disclaimer that outlines the limitations of the knowledge gained from the course to manage user expectations and prevent any claims of misrepresentation.

- Review the use of the phrase "embark on a learning journey that will enhance your skills and open doors to endless possibilities" for any misleading statements that could lead to legal implications.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

Review the following content. 
            

 Title: "Exploring the World of D

In [15]:
print(res.summary)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Unlock the world of artificial intelligence with DeepLearning.AI! Founded by the renowned Andrew Ng, this platform offers cutting-edge courses to help you master the intricacies of AI. Whether you're a beginner or an expert, DeepLearning.AI equips you with the knowledge and tools needed to thrive in this dynamic field. From neural networks to computer vision, embark on a transformative learning journey that will broaden your skill set and open up endless opportunities. Join the vibrant DeepLearning.AI community today to unleash the full potential of AI!
